# `apps_long.csv`

In [8]:
library(dplyr)
library(readr)
library(tidyr)

df <- read_csv("../output/apps_long.csv", show_col_types = FALSE)
results_df <- read_csv("https://raw.githubusercontent.com/petebrown/update-results/main/data/results_df.csv", show_col_types = FALSE)

#### Pivot apps_long to wide format mimicking book

In [9]:
df %>%
    filter(
        season == "1989/90"
    ) %>%
    arrange(
        season,
        game_no,
        player_name
    ) %>%
    pivot_wider(
        id_cols = c(season, game_no),
        names_from = player_name,
        values_from = shirt_no
    ) %>%
    left_join(results_df %>%
                  select(
                    season,
                    ssn_game_no,
                    opposition,
                    score,
                    generic_comp
                  ) %>%
                unique(),
              by = c("season" = "season", "game_no" = "ssn_game_no")
    ) %>%
    select(
        season, game_no, opposition, score, generic_comp,
        sort(names(.[3:length(.)]))
    )

season,game_no,opposition,score,generic_comp,Bauress,Bishop,Fairclough,Garnett,Harvey,⋯,McCarrick,McNab,Morrissey,Muir,Mungall,Nixon,Pike,Steel,Thomas,Vickers
<chr>,<dbl>,<chr>,<chr>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,⋯,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
1989/90,1,Fulham,2-1,Football League,NA,4,NA,NA,8,⋯,3,NA,7,10,NA,1,NA,9,11,6
1989/90,2,Preston North End,4-3,League Cup,NA,4,NA,NA,8,⋯,3,NA,7,10,NA,1,NA,NA,11,6
1989/90,3,Cardiff City,3-0,Football League,NA,4,NA,NA,8,⋯,3,NA,7,10,NA,1,NA,NA,11,6
1989/90,4,Preston North End,3-1,League Cup,NA,4,12,NA,8,⋯,3,NA,7,10,NA,1,NA,NA,11,6
1989/90,5,Reading,0-1,Football League,NA,4,12,NA,8,⋯,3,NA,7,10,NA,1,NA,NA,11,6
1989/90,6,Rotherham United,2-1,Football League,NA,4,12,NA,8,⋯,3,NA,7,10,NA,1,NA,NA,11,6
1989/90,7,Birmingham City,1-2,Football League,NA,4,12,NA,8,⋯,3,NA,7,10,NA,1,NA,NA,11,6
1989/90,8,Ipswich Town,1-0,League Cup,NA,14,NA,6,8,⋯,3,NA,7,10,NA,1,NA,12,11,NA
1989/90,9,Huddersfield Town,4-0,Football League,NA,14,NA,NA,8,⋯,3,NA,7,10,NA,1,NA,NA,11,6
